In [86]:
# Author: Joseph Riedy 
# Email: Riedyjo@bc.edu
# Stock Prediction: Takes a date and company ticker as inputs and uses machine learning to make 
# a price prediction based off the prior 30 days of closing stock prices. 
# DISCLAIMER: This is by no means meant to be financial advice. It was constructed merely as a person project.

import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor
import requests 
import xlsxwriter
#API sandbox token used to request information from IEX Cloud
#Data is randomized because this token is only for free use, real data is behind a paywall
from secrets import IEX_CLOUD_API_TOKEN

In [87]:
ydate=input("Please input the year of tomorrow's date in format YYYY:")
mdate=input("Please input the mont of tomorrow's date in format MM:")
ddate=input("Please input the day of tomorrow's date in format DD:")
tdate=ydate+mdate+ddate
symbol=input("Please input a stock ticker: ")
my_columns = ['Date', 'Stock Price']
final_dataframe = pd.DataFrame(columns = my_columns)
#irange='3m'

Please input the year of tomorrow's date in format YYYY:2009
Please input the mont of tomorrow's date in format MM:1
Please input the day of tomorrow's date in format DD:1
Please input a stock ticker: AAPL


In [88]:
nyear=int(ydate)
nmonth=int(mdate)
nday=int(ddate)
for j in range(30,0,-1):
    
    #API requests requires date in a specific YYYYMMDD format in order to work
    #In order to iterate, I had to come up with some way to automate the calendar math, 
    #this series of if and else statemetns allows the API reuqests to go across years 
    if nday-j<1:
        if nmonth==1:
            nyear=nyear-1
            nmonth=12
        else: 
            nmonth=nmonth-1
        if nmonth in (1,3,5,7,8,10,12):
            nday=31-j+nday
        elif nmonth in (4, 6, 9, 11):
            nday=30-j+nday
        else: 
            nday=28-j+nday
    else:
        nday=nday-j
    if nday<10:
        strnday="0" + str(nday)
    else:
        strnday=str(nday)
    if nmonth<10:
        strnmonth="0" + str(nmonth)
    else:
        strnmonth=str(nmonth)
    
    strnyear=str(nyear)
    strdate=strnyear+strnmonth+strnday
    #API URL is derived from IEX API docs found on the website 
    api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/chart/3m/{strdate}/?chartByDay=True&token={IEX_CLOUD_API_TOKEN}'
    
    #API URL is used to call data for the specific date of this iteration of the loop
    data=requests.get(api_url).json()
    
    #Is added to final_dataframe if stock info exists for specified day
    if (data):
        final_dataframe=final_dataframe.append(
            pd.Series(
                [
                    strdate,
                    data[0]['fClose']
                ],
                index=my_columns
            ),
        ignore_index=True
        )

In [90]:
#X and Y are arrays containing test data. x must be formatted as an array of arrays for KNeighborsRegressor
x=[]
y=[]
for i in range (0,len(final_dataframe)):
    x.append([i])
    y.append(int(final_dataframe.iloc[i][1]*100)) #only accepts integers in range thus must be multiplied by 100 in order to preserve cent count
    
#Forms a knn model out of data given from the mock API
#2 neighbors used because small number of variables mapped
model=KNeighborsRegressor(n_neighbors=2)
model.fit(x,y)
#Model is then used to predict the stock price of the user inputted date in USD
predictions = (model.predict([[-1]]))/100 
print(f'The predicted price on {mdate}/{ddate}/{ydate} is: ${predictions[0]}')

The predicted price on 1/1/2009 is: $3.12
